<p align="center" width="100%">
    <img width="40%" src="customer_support_icon.JPG"> 
</p>

A retail company is on a transformative journey, aiming to elevate their customer services through cutting-edge advancements in Speech Recognition and Natural Language Processing (NLP). As the machine learning engineer for this initiative, you are tasked with developing functionalities that not only convert customer support audio calls into text but also explore methodologies to extract insights from transcribed texts.

In this dynamic project, we leverage the power of `SpeechRecognition`, `Pydub`, and `spaCy` – three open-source packages that form the backbone of your solution. Your objectives are:
  - Transcribe a sample customer audio call, stored at `sample_customer_call.wav`, to showcase the power of open-source speech recognition technology.
  - Analyze sentiment, identify common named entities, and enhance user experience by searching for the most similar customer calls based on a given query from a subset of their pre-transcribed call data, stored at `customer_call_transcriptions.csv`.

This project is an opportunity to unlock the potential of machine learning to revolutionize customer support. Let's delve into the interplay between technology and service excellence.

In [11]:
!pip install SpeechRecognition
!pip install pydub
!pip install spacy
!python3 -m spacy download en_core_web_sm

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 134.3 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [12]:
# Import required libraries
import pandas as pd

import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

import speech_recognition as sr
from pydub import AudioSegment

import spacy

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/repl/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


## Is the audio compatible for future speech recognition modeling?

In [13]:
# Load the speech recognition model and recognize the audio
r = sr.Recognizer()
with sr.AudioFile('sample_customer_call.wav') as source:
    audio_data = r.record(source)
    transcribed_text = r.recognize_google(audio_data)

print(f"Transcribed Text: {transcribed_text}")

Transcribed Text: hello I'm experiencing an issue with your product I'd like to speak to someone about a replacement


In [14]:
# Load audio file with PyDub to analyze frame rate and channels
audio = AudioSegment.from_wav('sample_customer_call.wav')

# Get the frame rate and number of channels
frame_rate = audio.frame_rate
number_channels = audio.channels

print(f"Frame Rate: {frame_rate}")
print(f"Number of Channels: {number_channels}")

Frame Rate: 44100
Number of Channels: 1


## How many calls have a true positive sentiment?

In [15]:
# Load the dataset
data = pd.read_csv('customer_call_transcriptions.csv')

# Initialize VADER sentiment analyzer
sia = SentimentIntensityAnalyzer()

def analyze_sentiment(text):
    score = sia.polarity_scores(text)['compound']
    if score >= 0.05:
        return 'positive'
    elif score <= -0.05:
        return 'negative'
    else:
        return 'neutral'

# Apply sentiment analysis to the transcriptions
data['sentiment'] = data['text'].apply(analyze_sentiment)

# Calculate true positives for positive sentiments
true_positive = len(data[(data['sentiment'] == 'positive') & (data['sentiment_label'] == 'positive')])

print(f"True Positive Predictions: {true_positive}")

True Positive Predictions: 2


## What is the most frequently named entity across all of the transcriptions?

In [16]:
nlp = spacy.load("en_core_web_sm")

def find_most_freq_ent(transcriptions):
    # Process all transcriptions with spaCy's NER
    doc = nlp.pipe(transcriptions)

    ent_counter = {}
    for d in doc:
        for ent in d.ents:
            if ent.text in ent_counter:
                ent_counter[ent.text] += 1
            else:
                ent_counter[ent.text] = 1

    # Find the most frequent entity
    most_freq_ent, freq = max(ent_counter.items(), key=lambda x: x[1])

    return most_freq_ent, freq

# Apply function to transcriptions and store result in variable
transcriptions = data['text'].tolist()
most_freq_ent, freq = find_most_freq_ent(transcriptions)

print(f"Most frequently named entity: {most_freq_ent} (Frequency: {freq})")

Most frequently named entity: yesterday (Frequency: 15)


## Which call is the most similar to "wrong package delivery"?

In [17]:
# Import required library for similarity calculation
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Initialize TfidfVectorizer and encode transcriptions and query
transcriptions = data['text'].tolist()
vectorizer = TfidfVectorizer().fit_transform(transcriptions + ["wrong package delivery"])
query_vec = vectorizer[-1]

# Calculate cosine similarity between query and all transcriptions
similarity_scores = list(enumerate(cosine_similarity(query_vec, vectorizer[:len(transcriptions)]).flatten()))

# Get the index of the most similar transcription
most_similar_idx, _ = max(similarity_scores, key=lambda x: x[1])

# Save the most similar text as string variable
most_similar_text = data.iloc[most_similar_idx]['text']

print(f"Most similar call to 'wrong package delivery':\n{most_similar_text}")

Most similar call to 'wrong package delivery':
wrong package delivered
